# Solução - Projeto Automação Web - Busca de Preços

In [1]:
import pandas as pd
from selenium import webdriver

produtos = pd.read_excel(r'Produtos.xlsx')
produtos = produtos.fillna('-')
display(produtos)



,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,-,-
1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/Smart-LG-Intelig%C3%...,https://www.americanas.com.br/produto/49516150...,https://www.magazineluiza.com.br/smart-tv-uhd-...,2900,-,-


In [2]:
def transformar_texto(texto):
    return float(texto.replace('R$', '').replace('.', '').replace(',', '.'))

In [4]:
driver = webdriver.Chrome()
driver.set_window_position(-10000,0)

enviar_email = False
desconto_min = 0.2

for i, linha in produtos.iterrows():
    #pegar preço amazon
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element_by_id('priceblock_ourprice').text
        preco_amazon = transformar_texto(preco_amazon)
    except:
        try:
            preco_amazon = driver.find_element_by_css_selector('#olp-upd-new span.a-size-base').text
            preco_amazon = transformar_texto(preco_amazon)
        except:
            print('Produto {} não disponível na Amazon'.format(linha['Link Produto']))
            preco_amazon = linha['Preço Original'] * 3
    

    #pegar o preço da lame
    driver.get(linha['Lojas Americanas'])
    try:
        preco_lame = driver.find_element_by_class_name('src__BestPrice-lo2vta-5').text
        preco_lame = transformar_texto(preco_lame)
    except:
        print('Produto {} não disponível na Lojas Americanas'.format(linha['Link Produto']))
        preco_lame = linha['Preço Original'] * 3
        
    #pegar o preço da mglu
    driver.get(linha['Magazine Luiza'])
    try:
        preco_mglu = driver.find_element_by_class_name('price-template__text').text
        preco_mglu = transformar_texto(preco_mglu)
    except:
        print('Produto {} não disponível na Magazine Luiza'.format(linha['Link Produto']))
        preco_mglu = linha['Preço Original'] * 3

    preco_original = linha['Preço Original']
    
    lista_precos = [(preco_amazon, 'Amazon'), (preco_lame, 'Lojas Americanas'), (preco_mglu, 'Magazine Luiza'), (preco_original, 'Original')]
    
    lista_precos.sort()
    
    produtos.loc[i, 'Preço Atual'] = lista_precos[0][0]
    produtos.loc[i, 'Local'] = lista_precos[0][1]
    
    if lista_precos[0][0] <= preco_original*(1-desconto_min):
        enviar_email = True
    

Produto iPhone 11 Apple 64GB Preto não disponível na Amazon
Produto iPhone 11 Apple 64GB Preto não disponível na Lojas Americanas
Produto iPhone 11 Apple 64GB Preto não disponível na Magazine Luiza
Produto Smart TV LED 50'' LG Ultra HD 4K Thinq AI não disponível na Amazon
Produto Smart TV LED 50'' LG Ultra HD 4K Thinq AI não disponível na Lojas Americanas
Produto Smart TV LED 50'' LG Ultra HD 4K Thinq AI não disponível na Magazine Luiza


In [5]:
display(produtos)

,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,4783.87,Lojas Americanas
1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/Smart-LG-Intelig%C3%...,https://www.americanas.com.br/produto/49516150...,https://www.magazineluiza.com.br/smart-tv-uhd-...,2900,2459.90,Amazon


In [8]:
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

#salvar o arquivo
produtos.to_excel('Produtos.xlsx')
#enviar email

if enviar_email:
    #enviar email
    mail = outlook.CreateItem(0)
    mail.To = 'pythonimpressionador+compras@gmail.com'
    mail.Subject = f'Produto(s) Encontrado(s) com mais de {desconto_min:.0%} de Desconto'
    
    #filtrar a tabela de produtos
    tabela_filtrada = produtos.loc[produtos['Preço Atual'] <= produtos['Preço Original']*(1-desconto_min), :]
    
    mail.HTMLBody = f'<p>Esses sãos os produtos com mais de {desconto_min:.0%} de desconto</p>{tabela_filtrada.to_html()}'

    mail.Send()
    
#finalizando o código
print('Fim Análise')
driver.quit()

PermissionError: [Errno 13] Permission denied: 'Produtos.xlsx'